In [1]:
import pandas as pd
import os

In [2]:
# --- CONFIGURAÇÃO ---
# Caminho para a pasta onde você descompactou os CSVs da Olist
# Se estiver no Colab e subiu no drive, ajuste para '/content/drive/MyDrive/...'
DATA_PATH = '../data/' 


In [ ]:
# --- 1. CARREGAMENTO DOS DADOS (LEITURA) ---
print("Carregando datasets...")

# Tabela de Pedidos (O coração do sistema)
orders = pd.read_csv(os.path.join(DATA_PATH, 'olist_orders_dataset.csv'))

# Tabela de Itens (Detalhe do que foi comprado)
items = pd.read_csv(os.path.join(DATA_PATH, 'olist_order_items_dataset.csv'))

# Tabela de Produtos (Descrições, categorias)
products = pd.read_csv(os.path.join(DATA_PATH, 'olist_products_dataset.csv'))

# Tabela de Clientes (Localização)
customers = pd.read_csv(os.path.join(DATA_PATH, 'olist_customers_dataset.csv'))

# Tabela de Reviews (Avaliações para a parte de IA)
reviews = pd.read_csv(os.path.join(DATA_PATH, 'olist_order_reviews_dataset.csv'))

print(f"Dados carregados com sucesso!")
print(f"- Pedidos: {orders.shape}")
print(f"- Itens: {items.shape}")
print(f"- Produtos: {products.shape}")


Carregando datasets...
Dados carregados com sucesso!
- Pedidos: (99441, 8)
- Itens: (112650, 7)
- Produtos: (32951, 9)


In [4]:
# --- 2. MICROTRANSFORMAÇÃO (JOINS/MERGES) ---
print("\nIniciando unificação das tabelas...")

# Passo 1: Unir Pedidos com Itens
# (Um pedido pode ter vários itens, então o número de linhas vai aumentar igual ao dataset de itens)
df_merged = orders.merge(items, on='order_id', how='inner')



Iniciando unificação das tabelas...


In [5]:

# Passo 2: Adicionar informações do Produto
df_merged = df_merged.merge(products, on='product_id', how='left')


In [6]:

# Passo 3: Adicionar informações do Cliente
df_merged = df_merged.merge(customers, on='customer_id', how='left')


In [7]:

# Passo 4: Adicionar Reviews
# (Atenção: Um pedido pode ter mais de um review ou nenhum. Usamos left join para não perder vendas sem review)
df_merged = df_merged.merge(reviews, on='order_id', how='left')


In [8]:

# --- 3. LIMPEZA BÁSICA ---
# Selecionando apenas colunas úteis para o Dashboard e IA, renomeando para facilitar
df_final = df_merged[[
    'order_id', 
    'customer_unique_id',
    'order_status', 
    'order_purchase_timestamp', 
    'price', 
    'freight_value',
    'product_category_name', 
    'customer_city', 
    'customer_state',
    'review_score', 
    'review_comment_message' # Importante para o passo de GenAI
]]

# Convertendo data para formato datetime (essencial para séries temporais)
df_final['order_purchase_timestamp'] = pd.to_datetime(df_final['order_purchase_timestamp'])


C:\Users\luizm\AppData\Local\Temp\ipykernel_15536\124055000.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['order_purchase_timestamp'] = pd.to_datetime(df_final['order_purchase_timestamp'])


In [9]:
# --- 4. VALIDAÇÃO ---
print("\n--- Resultado Final (Amostra) ---")
print(df_final.info())
display(df_final.head())

# --- 5. SALVAR O RESULTADO (DATALAKE) ---
# Salvando em Parquet (muito mais rápido e leve que CSV para o Streamlit ler depois)
OUTPUT_PATH = '../data/olist_processed.parquet'
df_final.to_parquet(OUTPUT_PATH, index=False)
print(f"\nArquivo processado salvo em: {OUTPUT_PATH}")


--- Resultado Final (Amostra) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113314 entries, 0 to 113313
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  113314 non-null  object        
 1   customer_unique_id        113314 non-null  object        
 2   order_status              113314 non-null  object        
 3   order_purchase_timestamp  113314 non-null  datetime64[ns]
 4   price                     113314 non-null  float64       
 5   freight_value             113314 non-null  float64       
 6   product_category_name     111702 non-null  object        
 7   customer_city             113314 non-null  object        
 8   customer_state            113314 non-null  object        
 9   review_score              112372 non-null  float64       
 10  review_comment_message    47642 non-null   object        
dtypes: datetime64[ns](1), float64(

,order_id,customer_unique_id,order_status,order_purchase_timestamp,price,freight_value,product_category_name,customer_city,customer_state,review_score,review_comment_message
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,29.99,8.72,utilidades_domesticas,sao paulo,SP,4.0,"Não testei o produto ainda, mas ele veio corre..."
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,2018-07-24 20:41:37,118.70,22.76,perfumaria,barreiras,BA,4.0,Muito bom o produto.
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,delivered,2018-08-08 08:38:49,159.90,19.22,automotivo,vianopolis,GO,5.0,NaN
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,delivered,2017-11-18 19:28:06,45.00,27.20,pet_shop,sao goncalo do amarante,RN,5.0,O produto foi exatamente o que eu esperava e e...
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,delivered,2018-02-13 21:18:39,19.90,8.72,papelaria,santo andre,SP,5.0,NaN



Arquivo processado salvo em: ../data/olist_processed.parquet
